In [2]:
#git clone https://github.com/jwohlwend/boltz.git
cd boltz
git pull
git tag --list
SHORT_HASH='v1.0.0'
git checkout $SHORT_HASH
cd -

remote: Enumerating objects: 525, done.        
remote: Counting objects: 100% (195/195), done.        
remote: Compressing objects: 100% (81/81), done.        
remote: Total 525 (delta 154), reused 114 (delta 114), pack-reused 330 (from 3)        
Receiving objects: 100% (525/525), 277.59 KiB | 6.17 MiB/s, done.
Resolving deltas: 100% (252/252), completed with 24 local objects.
From https://github.com/jwohlwend/boltz
 * [new branch]      GH_181_chain_id_length -> origin/GH_181_chain_id_length
   2ba1f43..1f7acb1  main                   -> origin/main
 * [new tag]         v1.0.0                 -> v1.0.0
You are not currently on a branch.
Please specify which branch you want to rebase against.
See git-pull(1) for details.

    git pull <remote> <branch>

v0.1.0
v0.2.0
v0.2.1
v0.3.0
v0.3.1
v0.3.2
v0.4.0
v0.4.1
v1.0.0
Previous HEAD position was 9461d02 v0.4.1
HEAD is now at 34cf560 update docs
/Users/jjaenes/24.11_alphafold_on_euler/batch-infer/software/boltz


In [4]:
cd boltz
cp ../Dockerfile .
docker buildx build --platform=linux/amd64 --progress plain -t boltz-$SHORT_HASH --build-arg BASE_IMAGE=nvidia/cuda:12.1.1-cudnn8-runtime-ubuntu22.04 .
cd -

#0 building with "desktop-linux" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 1.27kB done
#1 WARN: InvalidDefaultArgInFrom: Default value for ARG ${BASE_IMAGE} results in empty or invalid base image name (line 3)
#1 WARN: InvalidDefaultArgInFrom: Default value for ARG ${BASE_IMAGE} results in empty or invalid base image name (line 20)
#1 DONE 0.0s

#2 [internal] load metadata for docker.io/nvidia/cuda:12.1.1-cudnn8-runtime-ubuntu22.04
#2 DONE 3.0s

#3 [internal] load .dockerignore
#3 transferring context: 2B done
#3 DONE 0.0s

#4 [internal] load build context
#4 transferring context: 2.35MB 0.0s done
#4 DONE 0.0s

#5 [builder 1/2] FROM docker.io/nvidia/cuda:12.1.1-cudnn8-runtime-ubuntu22.04@sha256:f4d8e1264366940438f0353da6f289c7bef069d993d111f8106086ccd18c4a30
#5 resolve docker.io/nvidia/cuda:12.1.1-cudnn8-runtime-ubuntu22.04@sha256:f4d8e1264366940438f0353da6f289c7bef069d993d111f8106086ccd18c4a30 done
#5 sha256:aece8493d

In [5]:
docker image ls

REPOSITORY              TAG       IMAGE ID       CREATED          SIZE
boltz-v1.0.0            latest    70bd951ebd97   38 seconds ago   9.54GB
mutfunc-a75f1ae-dirty   latest    7178eab23a7d   6 weeks ago      1.52GB
alphafold3-v3.0.1       latest    a4542dced624   3 months ago     9.13GB


In [6]:
mkdir -p images
docker save 70bd951ebd97 -o images/boltz-$SHORT_HASH.tar

In [7]:
SRC='./images/'
DEST='euler.ethz.ch:/cluster/project/beltrao/shared/alphafold3/images'
rsync -azv --progress $SRC $DEST

Transfer starting: 3 files
./
boltz-v0.4.1.tar
     9290540544 100%   22.17MB/s   00:06:39 (xfer#1, to-check=1/3)
boltz-v1.0.0.tar
     9575240704 100%   22.33MB/s   00:06:49 (xfer#2, to-check=2/3)

sent 10731999303 bytes  received 70 bytes  13270137 bytes/sec
total size is 18865781248  speedup is 1.76


In [8]:
ssh -T euler.ethz.ch << EOF
cd /cluster/project/beltrao/shared/alphafold3/images
time singularity build boltz-v1.0.0.sif docker-archive://boltz-v1.0.0.tar
EOF

      ____________________   ___
     /  ________   ___   /__/  /
    /  _____/  /  /  /  ___   /
   /_______/  /__/  /__/  /__/
   Eidgenoessische Technische Hochschule Zuerich
   Swiss Federal Institute of Technology Zurich
   -------------------------------------------------------------------------
                                                    E U L E R  C L U S T E R
                .-.
          .-'``(|||)
       ,`\ \    `-`.               88                         88
      /   \ '``-.   `              88                         88
    .-.  ,       `___:    88   88  88,888,  88   88  ,88888, 88888  88   88
   (:::) :        ___     88   88  88   88  88   88  88   88  88    88   88
    `-`  `       ,   :    88   88  88   88  88   88  88   88  88    88   88
      \   / ,..-`   ,     88   88  88   88  88   88  88   88  88    88   88
       `./ /    .-.`      '88888'  '88888'  '88888'  88   88  '8888 '88888'
          `-..-(   )
                `-`

   Documentation:  https://